#%pip install -U torch
#%pip install -U pandas
#%pip install -U matplotlib
#%pip install -U seaborn
#%pip install -U trl
#%pip install -U numpy
#%pip install -U peft
%pip install -U transformers
%pip install -U datasets
%pip install -U bitsandbytes #==0.41.1
%pip install -U huggingface_hub
#%pip install -U tqdm
%pip install -U codecarbon
%pip install -U ipywidgets
%pip install -U accelerate
%pip install -U vllm
%pip install -U wandb
#%pip install flash-attn --no-build-isolation
%pip install optimum
%pip install -U transformers

In [1]:
import numpy as np
import torch
import accelerate
from accelerate import infer_auto_device_map
#from datasets import load_dataset, Dataset
#from scipy.special import softmax
#from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
#from sklearn.metrics import accuracy_score, f1_score, log_loss, confusion_matrix
from transformers import  AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from huggingface_hub import notebook_login, login
#import json
import pandas as pd
import os
#from tqdm import tqdm
#import random
from codecarbon import EmissionsTracker, track_emissions
from time import time
import csv
from get_tokens import *
from vllm import LLM, SamplingParams

import wandb
import gc
import optimum

In [2]:
# Check if GPUs are available and move model to GPUs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
wandb.login(key=get_wandb_key())


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ubuntu/.netrc


True

In [4]:
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all

env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all


In [5]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

In [6]:
login(token=get_hf_token())
#notebook_login()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful


# Preparing the Test Data

In [7]:
# List of words that should be used to create sentences.

words = [
    "apple", "book", "car", "dog", "elephant", "forest", "guitar", "house", 
    "island", "jacket", "kangaroo", "lamp", "mountain", "notebook", "ocean", 
    "pencil", "queen", "river", "star", "tree", "umbrella", "village", 
    "window", "xylophone", "yacht", "zebra", "balloon", "camera", "desert", 
    "engine", "flower", "garden", "honey", "iceberg", "jungle", "kite", 
    "ladder", "moon", "nest", "octopus", "pirate", "quilt", "robot", "swan", 
    "telescope", "unicorn", "violin", "whale", "x-ray", "laptop"
]

In [8]:
# List of examples to be provided in the system prompt

example_sentences = [
    "Why did the bicycle fall over? Because it was two-tired!",
    "Why don't scientists trust atoms? Because they make up everything!",
    "Why did the scarecrow win an award? Because he was outstanding in his field!",
    "Why don't skeletons fight each other? They don't have the guts!",
    "Why did the computer go to the doctor? Because it had a virus!",
    "Why was the math book sad? Because it had too many problems!",
    "Why did the coffee file a police report? It got mugged!",
    "Why did the tomato turn red? Because it saw the salad dressing!",
    "Why don't eggs tell jokes? They might crack up!",
    "Why did the golfer bring two pairs of pants? In case he got a hole in one!",
    "Why do cows wear bells? Because their horns don't work!",
    "Why don't some couples go to the gym? Because some relationships don't work out!",
    "Why did the photo go to jail? It was framed!",
    "Why don't programmers like nature? It has too many bugs!",
    "Why did the bicycle stand up by itself? It was two-tired!",
    "Why did the music teacher need a ladder? To reach the high notes!",
    "Why did the cookie go to the doctor? Because it felt crummy!",
    "Why did the student eat his homework? Because his teacher told him it was a piece of cake!",
    "Why don't oysters donate to charity? Because they are shellfish!",
    "Why did the broom get a promotion? Because it swept the competition!",
    "Why don't we see elephants hiding in trees? Because they are so good at it!",
    "Why did the fish blush? Because it saw the ocean's bottom!",
    "Why did the barber win the race? Because he knew all the shortcuts!",
    "Why did the banana go to the doctor? Because it wasn't peeling well!",
    "Why don't dinosaurs talk? Because they are extinct!",
    "Why did the clock go back to school? To learn about time management!",
    "Why did the farmer win an award? Because he was outstanding in his field!",
    "Why did the astronaut break up with his girlfriend? He needed space!",
    "Why did the shoe go to the party alone? Because it didn't want to be a pair!",
    "Why did the tree go to the dentist? To get its roots checked!",
    "Why did the calendar go on a diet? It wanted to lose some days!",
    "Why did the stadium get hot? All the fans left!",
    "Why did the belt go to jail? It held up a pair of pants!",
    "Why did the cookie cry? Because its mom was a wafer too long!",
    "Why did the cow jump over the moon? To get to the milky way!",
    "Why did the skeleton go to the party alone? He had no body to go with!",
    "Why did the grape stop in the middle of the road? Because it ran out of juice!",
    "Why did the bee get married? Because he found his honey!",
    "Why did the soccer ball quit the team? It was tired of being kicked around!",
    "Why did the traffic light turn red? You would too if you had to change in the middle of the street!",
    "Why did the scarecrow become a successful neurosurgeon? He was outstanding in his field!",
    "Why did the chicken cross the playground? To get to the other slide!",
    "Why did the belt get a promotion? It was a cinch!",
    "Why did the teacher wear sunglasses? Because her students were so bright!",
    "Why did the clock get kicked out of class? It was tocking too much!",
    "Why did the frog call his insurance company? He had a jump in his car!",
    "Why did the keyboard get a speeding ticket? It had a problem with the space bar!",
    "Why did the painting go to art school? It wanted to brush up on its skills!",
    "Why did the scientist install a knocker on his door? He wanted to win the No-bell prize!",
    "Why did the tomato turn green? Because it was embarrassed to ketchup!",
    "Why did the pencil go to jail? It was caught in a sketchy situation!",
    "Why did the music note need a loan? It needed some major funding!",
    "Why did the fisherman put peanut butter into the sea? To go with the jellyfish!",
    "Why did the bicycle bring a map? Because it didn't want to get lost on its wheel-y big adventure!",
    "Why did the pizza go to the party? It wanted to slice up the dance floor!",
    "Why did the phone sit on a bench? It wanted to recharge its batteries!",
    "Why did the blanket get arrested? It was covering up a crime!",
    "Why did the chef go to jail? Because he beat the eggs and whipped the cream!",
    "Why did the sandwich go to the beach? To get a little bologna-sun!",
    "Why did the banker switch careers? He lost interest!",
    "Why did the shovel go to therapy? It had too much dirt on its mind!",
    "Why did the soccer player bring string to the game? So he could tie the score!",
    "Why did the alarm clock break up with the pillow? It couldn't handle the pressure!",
    "Why did the frog take the bus to work? His car got toad away!",
    "Why did the dentist become a gardener? He wanted to brush up on his roots!",
    "Why did the light bulb fail his test? He wasn't too bright!",
    "Why did the ocean break up with the shore? It needed some space to tide things over!",
    "Why did the shoe store close down? It lost its sole!",
    "Why did the banana go out with the prune? Because it couldn't find a date!",
    "Why did the carpenter become a musician? He wanted to nail every note!"
]


In [9]:
def prepare_prompts(word_list, runs=1, num_examples=1, multiply_by=1):
    system_message = f"""
    You are an AI assistant designed to tell jokes for a given word.
    Generate {num_examples*multiply_by} short joke(s) that are exactly 2 short sentences long. Do not include any follow-up questions or explanations.

    For example, a joke for the word "yogurt":
    'Why did the yogurt go to the art gallery? Because it wanted to be cultured!'

    Here are a few further examples: 
    - 'Why did the bicycle fall over? Because it was two-tired!'
    - 'Why don't scientists trust atoms? Because they make up everything!'

    
    Be as close as possible to the example jokes!
    Respond with only the {num_examples*multiply_by} joke(s) itself. Do not include any introductory phrases.

    """

    prompts = []
    
    for r in range(runs):

        for word in word_list:
            user_message = "Question: " + f'Now, tell me {num_examples*multiply_by} short joke(s) for the word: "{word}"\n' + " Answer:"
            
            messages = [
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message},
            ]
            prompt = tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
            )

            prompts.append(prompt)
    
    
    return prompts

# vLLM

## Creating the Model

In [10]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.bos_token

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
# Create an LLM.
llm = LLM(model=model_name,
          tensor_parallel_size=4, # For 4 GPUs
          #pipeline_parallel_size=2, # Can be set to 4
          #max_num_batched_tokens=8192, # Could be increased for faster throughput
          #max_num_seqs=512, #Increase if  application can benefit from processing more sequences simultaneously. Start with 512 and adjust.
          dtype='bfloat16', # Ensure fp16 precision
          #max_parallel_loading_workers=24, #Set to the number of CPU cores divided by 2.
          #trust_remote_code=True)
)
# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.

/opt/conda/envs/pytorch/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

2024-06-15 18:04:38,364	INFO worker.py:1753 -- Started a local Ray instance.


INFO 06-15 18:04:39 config.py:623] Defaulting to use mp for distributed inference
INFO 06-15 18:04:39 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B-Instruct)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

INFO 06-15 18:04:59 model_runner.py:160] Loading model weights took 3.7417 GB
(VllmWorkerProcess pid=36340) INFO 06-15 18:05:00 model_runner.py:160] Loading model weights took 3.7417 GB
(VllmWorkerProcess pid=36339) INFO 06-15 18:05:00 model_runner.py:160] Loading model weights took 3.7417 GB
(VllmWorkerProcess pid=36341) INFO 06-15 18:05:00 model_runner.py:160] Loading model weights took 3.7417 GB
INFO 06-15 18:05:05 distributed_gpu_executor.py:56] # GPU blocks: 29736, # CPU blocks: 8192
(VllmWorkerProcess pid=36340) INFO 06-15 18:05:10 model_runner.py:889] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=36340) INFO 06-15 18:05:10 model_runner.py:893] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You 

: 

## Testing the Model

In [12]:
def query_model_vllm(prompt_list, temperature=0.8, top_p=0.95, max_length=48):

    # Create a sampling params object.
    sampling_params = SamplingParams(temperature=temperature, top_p=top_p, max_tokens= max_length)

    # Start Timer for Inference
    start_time = time()

    outputs = llm.generate(prompt_list, sampling_params)

    # End Timer for Inference
    end_time = time()

    ttime = end_time-start_time

    return outputs, ttime

In [14]:
runs = 20
num_prompts = len(words)
total_prompts = runs * num_prompts

example_counts = [1, 5, 20, 30]
multiply = [1,2,3]

for multiply_by in multiply:

    for num_examples in example_counts:

        total_input_tok = 0
        total_output_tok = 0

        print("="*10 + f" INFERENCE TEST with {num_examples * multiply_by} EXAMPLES " + "="*10 + 
        "\n\n" + 
        f"""
        Starting Test with {runs} Runs and {num_prompts} Prompts / Run. \n
        Total Prompts: {total_prompts}\n\n
        """)

        name=f"vLLM_Inference_{total_prompts}_prompts_output_tok_{num_examples*multiply_by}_examples"

        prompts = prepare_prompts(words, runs=runs, num_examples=num_examples, multiply_by=multiply_by)


        wandb.init(
            # set the wandb project where this run will be logged
            project="Inference_Framework_Comparison",

            # track hyperparameters and run metadata
            config={
            "runs": runs,
            "num_prompts": num_prompts,
            "total_prompts": total_prompts,
            "framework": 'vLLM',
            "model": model_name,
            },

            name=name,
        )

        tracker = EmissionsTracker(save_to_file=True, project_name=f"{name}-llama3-8B", log_level="error", pue = 1.22, output_file=f"emissions_output_tok_vllm.csv")
        tracker.start()


        outputs, ttime = query_model_vllm(prompts, max_length=48*num_examples*multiply_by)

        emissions: float = tracker.stop()



        for output in outputs: 


            # Extracting information
            prompt = output.prompt
            generated_text = output.outputs[0].text
            input_tokens = output.prompt_token_ids
            output_tokens = output.outputs[0].token_ids
            num_input_tokens = len(input_tokens)
            num_output_tokens = len(output_tokens)

            # Updating cumulative counts
            total_input_tok += num_input_tokens
            total_output_tok += num_output_tokens


        # Calculate averages
        avg_time_per_prompt = (ttime / total_prompts)*1000
        avg_toks_per_sec = total_output_tok/ttime
        avg_input_tokens = total_input_tok / total_prompts
        avg_output_tokens = total_output_tok / total_prompts

        em_i = emissions/total_input_tok *1_000_000
        em_o = emissions/total_output_tok *1_000_000
        em_p = emissions/total_prompts *10_000

        print("="*15 + f" RESULTS for {name} " + "="*15 + 
            "\n\n" + 
            f"""
            Finished {runs} Runs with {num_prompts} Prompts/Run.\n\n
            Total Time: {ttime:.2f}s, AVG/Prompt: {avg_time_per_prompt:.2f}ms\n\n
            Average tokens per second: {avg_toks_per_sec:.2f}\n\n
            Total Prompts: {total_prompts}\n
            Total Input Tokens: {total_input_tok}, AVG/Prompt: {avg_input_tokens}\n
            Total Output Tokens: {total_output_tok}, AVG/Prompt: {avg_output_tokens}\n
            """ + 
            
            "-"*50 + "\n" +
            
            f"""
            Total Inference Emissions: {emissions:.3f}kg CO₂eq\n\n
            Emissions / 1.000.000 Input Tokens: {em_i:.3f}kg CO₂eq\n
            Emissions / 1.000.000 Output Tokens: {em_o:.3f}kg CO₂eq\n
            Emissions / 10.000 Prompts: {em_p:.3f}kg CO₂eq\n

            """
            )

        wandb.log({"Total Time": ttime,
            "AVG. Time / Prompt": avg_time_per_prompt,
                    "AVG. Tokens / Second": avg_toks_per_sec,
                    "AVG. Input Tokens": avg_input_tokens,
                    "AVG. Output Tokens": avg_output_tokens,
                    "Total Emissions": emissions,
                    "Emissions / 1.000.000 Input Tokens": em_i,
                    "Emissions / 1.000.000 Output Tokens": em_o,
                    "Emissions / 10.000 Prompts": em_p,
                    })

        wandb.finish()

        # Save results to a CSV file
        results = [
            ["Runs", runs],
            ["Prompts / Run", num_prompts],
            ["Total Prompts", total_prompts],
            ["Total Time", ttime], 
            ["AVG. Time / Prompt", avg_time_per_prompt],
            ["AVG. Tokens / Second", avg_toks_per_sec],
            ["Total Input Tokens", total_input_tok],
            ["AVG. Input Tokens / Prompt", avg_input_tokens],
            ["Total Output Tokens", total_output_tok],
            ["AVG. Output Tokens / Prompt", avg_output_tokens],
            ["Total Emissions", emissions],
            ["Emissions / 1.000.000 Input Tokens", em_i],
            ["Emissions / 1.000.000 Output Tokens", em_o],
            ["Emissions / 10.000 Prompts", em_p]
        ]

        # Ensure the directory exists
        output_file_path = f"emission_data/vllm_output_tok_{model_name}_emission_data_{num_examples*multiply_by}_examples.csv"
        os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

        with open(output_file_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Metric", "Value"])
            writer.writerows(results)

        print(f"Results saved to {output_file_path}\n\n")

========== INFERENCE TEST with 1 EXAMPLES ==========


        Starting Test with 20 Runs and 50 Prompts / Run. 

        Total Prompts: 1000


        


Processed prompts: 100%|██████████| 1000/1000 [00:48<00:00, 20.55it/s, Generation Speed: 373.46 toks/s]
/opt/conda/envs/pytorch/lib/python3.10/site-packages/codecarbon/output_methods/file.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(data.values)])])


=============== RESULTS for vLLM_Inference_1000_prompts_output_tok_1_examples ===============


            Finished 20 Runs with 50 Prompts/Run.


            Total Time: 49.20s, AVG/Prompt: 49.20ms


            Average tokens per second: 369.35


            Total Prompts: 1000

            Total Input Tokens: 186660, AVG/Prompt: 186.66

            Total Output Tokens: 18173, AVG/Prompt: 18.173

            --------------------------------------------------

            Total Inference Emissions: 0.003kg CO₂eq


            Emissions / 1.000.000 Input Tokens: 0.019kg CO₂eq

            Emissions / 1.000.000 Output Tokens: 0.192kg CO₂eq

            Emissions / 10.000 Prompts: 0.035kg CO₂eq


            


AVG. Input Tokens,▁
AVG. Output Tokens,▁
AVG. Time / Prompt,▁
AVG. Tokens / Second,▁
Emissions / 1.000.000 Input Tokens,▁
Emissions / 1.000.000 Output Tokens,▁
Emissions / 10.000 Prompts,▁
Total Emissions,▁
Total Time,▁
AVG. Input Tokens,186.66
AVG. Output Tokens,18.173


Results saved to emission_data/vllm_output_tok_meta-llama/Meta-Llama-3-8B-Instruct_emission_data_1_examples.csv


========== INFERENCE TEST with 5 EXAMPLES ==========


        Starting Test with 20 Runs and 50 Prompts / Run. 

        Total Prompts: 1000


        


...

Processed prompts: 100%|██████████| 1000/1000 [24:06<00:00,  1.45s/it, Generation Speed: 1241.50 toks/s]


=============== RESULTS for vLLM_Inference_1000_prompts_output_tok_90_examples ===============


            Finished 20 Runs with 50 Prompts/Run.


            Total Time: 1447.27s, AVG/Prompt: 1447.27ms


            Average tokens per second: 1241.17


            Total Prompts: 1000

            Total Input Tokens: 186660, AVG/Prompt: 186.66

            Total Output Tokens: 1796304, AVG/Prompt: 1796.304

            --------------------------------------------------

            Total Inference Emissions: 0.110kg CO₂eq


            Emissions / 1.000.000 Input Tokens: 0.591kg CO₂eq

            Emissions / 1.000.000 Output Tokens: 0.061kg CO₂eq

            Emissions / 10.000 Prompts: 1.103kg CO₂eq


            


AVG. Input Tokens,▁
AVG. Output Tokens,▁
AVG. Time / Prompt,▁
AVG. Tokens / Second,▁
Emissions / 1.000.000 Input Tokens,▁
Emissions / 1.000.000 Output Tokens,▁
Emissions / 10.000 Prompts,▁
Total Emissions,▁
Total Time,▁
AVG. Input Tokens,186.66
AVG. Output Tokens,1796.304


Results saved to emission_data/vllm_output_tok_meta-llama/Meta-Llama-3-8B-Instruct_emission_data_90_examples.csv




2024-06-11 16:02:37,909	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::RayWorkerWrapper.execute_method() (pid=65465, ip=172.31.17.49, actor_id=137f8ff7b0b180719fc9ed0d01000000, repr=<vllm.executor.ray_utils.RayWorkerWrapper object at 0x7fc25393d900>)
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/vllm/worker/worker_base.py", line 149, in execute_method
    raise e
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/vllm/worker/worker_base.py", line 140, in execute_method
    return executor(*args, **kwargs)
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/vllm/worker/worker.py", line 286, in start_worker_execution_loop
    while self._execute_model_non_driver():
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/vllm/worker/worker.py", line 295, 

# Note: Idle Performance

- In idle each L4 GPU needs about 27W to store its maximum capacity in VRAM. 
- In full idle with empty VRAM each L4 needs about 16W

In [15]:
import time

tracker = EmissionsTracker(save_to_file=True, project_name=f"idle_vllm_output", log_level="error", pue = 1.22, output_file=f"emissions_output_tok_vllm.csv")
tracker.start()


time.sleep(60)

tracker.stop()

0.003014534340176567

# Huggingface pyTorch - Optimized Flash Attention 2

In [10]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.bos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
# Load the model with flash attention 2
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    #attn_implementation="flash_attention_2", #FlashAttention only supports Ampere GPUs or newer.
)
model.config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
# Initialize the pipeline
text_generation_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer, device_map="auto", batch_size=64, num_workers=48)

In [13]:
def query_model_pipeline(prompt_list, temperature=0.8, top_p=0.95, max_length=48):

    # Start Timer for Inference
    start_time = time()

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = text_generation_pipeline(
        prompt_list,
        do_sample=True,
        top_p=top_p,
        temperature=temperature,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        #return_tensors=True,
        #return_text = True,
        pad_token_id=model.config.eos_token_id
    )

    # End Timer for Inference
    end_time = time()

    ttime = end_time-start_time

    return outputs, ttime

In [14]:
total_input_tok = 0
total_output_tok = 0

runs = 10
num_prompts = len(words)
total_prompts = runs * num_prompts

name=f"pytorch_pipeline_Inference_{total_prompts}_prompts"


prompts = prepare_prompts(words, runs=runs, num_examples=1, multiply_by=1)


wandb.init(
    # set the wandb project where this run will be logged
    project="Inference_Framework_Comparison",

    # track hyperparameters and run metadata
    config={
    "runs": runs,
    "num_prompts": num_prompts,
    "total_prompts": total_prompts,
    "framework": 'pyTorch - Transformers Pipeline',
    "model": model_name,
    },

    name=name,
)

print('\n --->>> Starting Test Run <<<--- \n')

tracker = EmissionsTracker(save_to_file=True, project_name=f"{name}-llama3-8B", log_level="error", pue = 1.22, output_file=f"emissions_{name}.csv")
tracker.start()


outputs, ttime = query_model_pipeline(prompts)

emissions: float = tracker.stop()
    
print(f'\n Finished in: {ttime:.2f}s!\n\n')

  
    
for idx, output in enumerate(outputs): 
    
    # Extracting information
    prompt = prompts[idx]
    generated_text = output[0]['generated_text']
    
    input_tokens = tokenizer.encode(prompt)
    input_token_count = len(input_tokens)
    
    output_tokens = tokenizer.encode(generated_text)
    num_output_tokens = len(output_tokens)

    total_input_tok += input_token_count
    total_output_tok += num_output_tokens


# Calculate averages
avg_time_per_prompt = (ttime / total_prompts)*1000
avg_toks_per_sec = total_output_tok/ttime
avg_input_tokens = total_input_tok / total_prompts
avg_output_tokens = total_output_tok / total_prompts

em_i = emissions/total_input_tok *1_000_000
em_o = emissions/total_output_tok *1_000_000
em_p = emissions/total_prompts *10_000

print("="*15 + f" RESULTS for {name} " + "="*15 + 
    "\n\n" + 
    f"""
    Finished {runs} Runs with {num_prompts} Prompts/Run.\n\n
    Total Time: {ttime:.2f}s, AVG/Prompt: {avg_time_per_prompt:.2f}ms\n\n
    Average tokens per second: {avg_toks_per_sec:.2f}\n\n
    Total Prompts: {total_prompts}\n
    Total Input Tokens: {total_input_tok}, AVG/Prompt: {avg_input_tokens}\n
    Total Output Tokens: {total_output_tok}, AVG/Prompt: {avg_output_tokens}\n
    """ + 
    
    "-"*50 + "\n" +
    
    f"""
    Total Inference Emissions: {emissions:.3f}kg CO₂eq\n\n
    Emissions / 1.000.000 Input Tokens: {em_i:.3f}kg CO₂eq\n
    Emissions / 1.000.000 Output Tokens: {em_o:.3f}kg CO₂eq\n
    Emissions / 10.000 Prompts: {em_p:.3f}kg CO₂eq\n

    """
    )

wandb.log({"Total Time": ttime,
    "AVG. Time / Prompt": avg_time_per_prompt,
              "AVG. Tokens / Second": avg_toks_per_sec,
              "AVG. Input Tokens": avg_input_tokens,
              "AVG. Output Tokens": avg_output_tokens,
              "Total Emissions": emissions,
              "Emissions / 1.000.000 Input Tokens": em_i,
              "Emissions / 1.000.000 Output Tokens": em_o,
              "Emissions / 10.000 Prompts": em_p,
              })

wandb.finish()


 --->>> Starting Test Run <<<--- 



/opt/conda/envs/pytorch/lib/python3.10/site-packages/codecarbon/output_methods/file.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(data.values)])])



 Finished in: 74.87s!


=============== RESULTS for pytorch_pipeline_Inference_500_prompts ===============


    Finished 10 Runs with 50 Prompts/Run.


    Total Time: 74.87s, AVG/Prompt: 149.73ms


    Average tokens per second: 105.66


    Total Prompts: 500

    Total Input Tokens: 93330, AVG/Prompt: 186.66

    Total Output Tokens: 7910, AVG/Prompt: 15.82

    --------------------------------------------------

    Total Inference Emissions: 0.005kg CO₂eq


    Emissions / 1.000.000 Input Tokens: 0.049kg CO₂eq

    Emissions / 1.000.000 Output Tokens: 0.573kg CO₂eq

    Emissions / 10.000 Prompts: 0.091kg CO₂eq


    


AVG. Input Tokens,▁
AVG. Output Tokens,▁
AVG. Time / Prompt,▁
AVG. Tokens / Second,▁
Emissions / 1.000.000 Input Tokens,▁
Emissions / 1.000.000 Output Tokens,▁
Emissions / 10.000 Prompts,▁
Total Emissions,▁
Total Time,▁
AVG. Input Tokens,186.66
AVG. Output Tokens,15.82


In [39]:
torch.cuda.empty_cache()

# Adding TOrch Compile & Static kv-cache

In [ ]:
# torch.compile requires static kv-cache
model.generation_config.cache_implementation = "static"

In [ ]:
compiled_model = torch.compile(model, mode="max-autotune", fullgraph=True)

In [ ]:
# Initialize the pipeline
text_generation_pipeline = pipeline('text-generation', model=compiled_model, tokenizer=tokenizer, device_map="auto", batch_size=64, num_workers=48)

In [ ]:
def query_model_pipeline(prompt_list, temperature=0.8, top_p=0.95, max_length=48):

    # Start Timer for Inference
    start_time = time()

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    outputs = text_generation_pipeline(
        prompt_list,
        do_sample=True,
        top_p=top_p,
        temperature=temperature,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        #return_tensors=True,
        #return_text = True,
        pad_token_id=model.config.eos_token_id
    )

    # End Timer for Inference
    end_time = time()

    ttime = end_time-start_time

    return outputs, ttime

In [ ]:
total_input_tok = 0
total_output_tok = 0

runs = 1
num_prompts = len(words)
total_prompts = runs * num_prompts

name=f"Kaggle_pytorch_pipeline_Inference_{total_prompts}_prompts"


prompts = prepare_prompts(words, runs=runs, num_examples=1, multiply_by=1)


wandb.init(
    # set the wandb project where this run will be logged
    project="Inference_Framework_Comparison",

    # track hyperparameters and run metadata
    config={
    "runs": runs,
    "num_prompts": num_prompts,
    "total_prompts": total_prompts,
    "framework": 'pyTorch - Transformers Pipeline',
    "model": model_name,
    },

    name=name,
)

print('\n --->>> Starting Test Run <<<--- \n')

tracker = EmissionsTracker(save_to_file=True, project_name=f"{name}-llama3-8B", log_level="error", pue = 1.22, output_file=f"emissions_{name}.csv")
tracker.start()

with torch.no_grad():
    outputs, ttime = query_model_pipeline(prompts)

emissions: float = tracker.stop()
    
print(f'\n Finished in: {ttime:.2f}s!\n\n')

  
    
for idx, output in enumerate(outputs): 
    
    # Extracting information
    prompt = prompts[idx]
    generated_text = output[0]['generated_text']
    
    input_tokens = tokenizer.encode(prompt)
    input_token_count = len(input_tokens)
    
    output_tokens = tokenizer.encode(generated_text)
    num_output_tokens = len(output_tokens)

    total_input_tok += input_token_count
    total_output_tok += num_output_tokens


# Calculate averages
avg_time_per_prompt = (ttime / total_prompts)*1000
avg_toks_per_sec = total_output_tok/ttime
avg_input_tokens = total_input_tok / total_prompts
avg_output_tokens = total_output_tok / total_prompts

em_i = emissions/total_input_tok *1_000_000
em_o = emissions/total_output_tok *1_000_000
em_p = emissions/total_prompts *10_000

print("="*15 + f" RESULTS for {name} " + "="*15 + 
    "\n\n" + 
    f"""
    Finished {runs} Runs with {num_prompts} Prompts/Run.\n\n
    Total Time: {ttime:.2f}s, AVG/Prompt: {avg_time_per_prompt:.2f}ms\n\n
    Average tokens per second: {avg_toks_per_sec:.2f}\n\n
    Total Prompts: {total_prompts}\n
    Total Input Tokens: {total_input_tok}, AVG/Prompt: {avg_input_tokens}\n
    Total Output Tokens: {total_output_tok}, AVG/Prompt: {avg_output_tokens}\n
    """ + 
    
    "-"*50 + "\n" +
    
    f"""
    Total Inference Emissions: {emissions:.3f}kg CO₂eq\n\n
    Emissions / 1.000.000 Input Tokens: {em_i:.3f}kg CO₂eq\n
    Emissions / 1.000.000 Output Tokens: {em_o:.3f}kg CO₂eq\n
    Emissions / 10.000 Prompts: {em_p:.3f}kg CO₂eq\n

    """
    )

wandb.log({"Total Time": ttime,
    "AVG. Time / Prompt": avg_time_per_prompt,
              "AVG. Tokens / Second": avg_toks_per_sec,
              "AVG. Input Tokens": avg_input_tokens,
              "AVG. Output Tokens": avg_output_tokens,
              "Total Emissions": emissions,
              "Emissions / 1.000.000 Input Tokens": em_i,
              "Emissions / 1.000.000 Output Tokens": em_o,
              "Emissions / 10.000 Prompts": em_p,
              })

wandb.finish()

# Hugging Face Optimum

# llama.cpp

# Hugging Face Transformers - Not Optimized

In [19]:
del llm
torch.cuda.empty_cache()
gc.collect()

83558

In [8]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.bos_token

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    #torch_dtype=torch.float16,
)
model.config.pad_token_id = tokenizer.pad_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Test the Model

# Approach with Pipeline

In [11]:
# Initialize the pipeline
text_generation_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer, device_map="auto")

In [12]:
def query_model(word, temperature=0.6, max_length=48, num_examples=1, multiply_by=1):
    start_time = time()
    

    system_message = f"""
    You are an AI assistant designed to tell jokes for a given word.
    Generate {num_examples*multiply_by} short joke(s) that are exactly 2 short sentences long. Do not include any follow-up questions or explanations.

    For example, a joke for the word "yogurt":
    'Why did the yogurt go to the art gallery? Because it wanted to be cultured!'

    Here are a few further examples: 
    - 'Why did the bicycle fall over? Because it was two-tired!'
    - 'Why don't scientists trust atoms? Because they make up everything!'

    
    Be as close as possible to the example jokes!
    Respond with only the {num_examples*multiply_by} joke(s) itself. Do not include any introductory phrases.

    """
    
    user_message = "Question: " + f'Now, tell me {num_examples*multiply_by} short joke(s) for the word: "{word}"\n' + " Answer:"
    
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]
    prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    # Tokenize the prompt to count input tokens
    input_tokens = tokenizer.encode(prompt)
    input_token_count = len(input_tokens)
    
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = text_generation_pipeline(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=model.config.eos_token_id
    )
    answer = sequences[0]['generated_text']
    
    # Tokenize the answer to count output tokens
    output_tokens = tokenizer.encode(answer)
    output_token_count = len(output_tokens)
    
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return (
        user_message + "\n\n" + answer + "\n\n" + ttime,
        input_token_count,
        output_token_count
    )

In [13]:
avg_time = 10
avg_in_tok = 103
avg_out_tok = 30

model_name = "outputtest-meta-llama/Meta-Llama-3-8B-Instruct-OutputTest"

#avg_time = 4.5
#avg_in_tok = 103
#avg_out_tok = 25

# Number of runs
n = 3 # Change to 5 for all the other tests
num_words = len(words)
total_prompts = n*num_words

example_counts = [1, 5, 10, 20, 30]
#multiply = [1, 3, 5] # 5 did not work yet, because GPU 1 had too much in memory. Maybe try with other GPU map

#num_examples = 70
multiply_by = 1

for num_examples in example_counts:

    total_input_tokens = 0
    total_output_tokens = 0
    max_length = num_examples * multiply_by * 30

    print("="*10 + f" INFERENCE TEST with {num_examples * multiply_by} EXAMPLES " + "="*10 + 
        "\n\n" + 
        f"""
        Starting Test with {n} Runs and {num_words} Words/Run. \n
        Total Prompts: {total_prompts}\n\n
        Expected Runtime: {(total_prompts*avg_time)/60}min \n
        Expected Input Tokens: {avg_in_tok*total_prompts} \n
        Expected Output Tokens: {avg_out_tok*total_prompts} \n\n
        """)

    tracker = EmissionsTracker(project_name=f'{model_name}-emissiondata-{num_examples * multiply_by}-examples', log_level="error", pue = 1.22)
    tracker.start()

    # Run the function for each word in the array
    for i in range(n):
        print("="*25 + f" Run {i+1} " + "="*25)
        print("\n")
        for idx, word in enumerate(words):
            print("-"*15 + f" Run {i+1} - Word {idx+1} " + "-"*15)
            output, input_token_count, output_token_count = query_model(word, max_length=max_length, num_examples=num_examples, temperature=0.2, multiply_by=multiply_by)
            
            total_input_tokens += input_token_count
            total_output_tokens += output_token_count
            
            print(output)
            print("\n")
            print(f"Input tokens: {input_token_count}, Output tokens: {output_token_count}")
            print("\n")
        print("\n")
    
    emissions: float = tracker.stop()

    em_i = emissions/total_input_tokens
    em_o = emissions/total_output_tokens
    em_p = emissions/total_prompts


    print("="*15 + f" RESULTS for {num_examples * multiply_by} EXAMPLES " + "="*15 + 
        "\n\n" + 
        f"""
        Finished {n} Runs with {num_words} Words/Run.\n\n
        Total Prompts: {total_prompts}\n
        Total Input Tokens: {total_input_tokens}, AVG/Prompt: {total_input_tokens/total_prompts}\n
        Total Output Tokens: {total_output_tokens}, AVG/Prompt: {total_output_tokens/total_prompts}\n
        """ + 
        
        "-"*50 + "\n" +
        
        f"""
        Total Inference Emissions: {emissions:.3f}kg CO₂eq\n\n
        Emissions / 1.000.000 Input Tokens: {em_i * 1000000:.3f}kg CO₂eq\n
        Emissions / 1.000.000 Output Tokens: {em_o * 1000000:.3f}kg CO₂eq\n
        Emissions / 1.000 Prompts: {em_p * 1000:.3f}kg CO₂eq\n

        """
        )

    # Save results to a CSV file
    results = [
        ["Runs", n],
        ["Words per Run", num_words],
        ["Total Prompts", total_prompts],
        ["Total Input Tokens", total_input_tokens],
        ["Avg Input Tokens per Prompt", round(total_input_tokens / total_prompts, 3)],
        ["Total Output Tokens", total_output_tokens],
        ["Avg Output Tokens per Prompt", round(total_output_tokens / total_prompts, 3)],
        ["Total Emissions (kg CO₂eq)", round(emissions, 3)],
        ["Emissions per Million Input Tokens (kg CO₂eq)", round(em_i * 1000000, 3)],
        ["Emissions per Million Output Tokens (kg CO₂eq)", round(em_o * 1000000, 3)],
        ["Emissions per Thousand Prompts (kg CO₂eq)", round(em_p * 1000, 3)]
    ]

    # Ensure the directory exists
    output_file_path = f'{model_name}-emissiondata-{num_examples * multiply_by}-examples.csv'
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

    with open(output_file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Metric", "Value"])
        writer.writerows(results)

    print(f"Results saved to {output_file_path}")

========== INFERENCE TEST with 1 EXAMPLES ==========


        Starting Test with 3 Runs and 50 Words/Run. 

        Total Prompts: 150


        Expected Runtime: 25.0min 

        Expected Input Tokens: 15450 

        Expected Output Tokens: 4500 


        
========================= Run 1 =========================


--------------- Run 1 - Word 1 ---------------
Question: Now, tell me 1 short joke(s) for the word: "apple"
 Answer:

Why did the apple join the gym? Because it wanted to get some core strength!

Total time: 3.52 sec.


Input tokens: 186, Output tokens: 18


--------------- Run 1 - Word 2 ---------------
Question: Now, tell me 1 short joke(s) for the word: "book"
 Answer:

Why did the book go to therapy? Because it had a bad chapter in its life!

Total time: 2.48 sec.


Input tokens: 186, Output tokens: 19


--------------- Run 1 - Word 3 ---------------
Question: Now, tell me 1 short joke(s) for the word: "car"
 Answer:

Why did the car go to the doctor? Because it was

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Question: Now, tell me 1 short joke(s) for the word: "jacket"
 Answer:

Why did the jacket go to therapy? Because it was feeling a little worn out!

Total time: 2.38 sec.


Input tokens: 187, Output tokens: 18


--------------- Run 1 - Word 11 ---------------
Question: Now, tell me 1 short joke(s) for the word: "kangaroo"
 Answer:

Why did the kangaroo quit his job? Because he was hopping mad about the working conditions!

Total time: 2.61 sec.


Input tokens: 188, Output tokens: 20


--------------- Run 1 - Word 12 ---------------
Question: Now, tell me 1 short joke(s) for the word: "lamp"
 Answer:

Why did the lamp go to therapy? Because it was feeling a little dim!

Total time: 2.23 sec.


Input tokens: 186, Output tokens: 17


--------------- Run 1 - Word 13 ---------------
Question: Now, tell me 1 short joke(s) for the word: "mountain"
 Answer:

Why did the mountain go to the party? Because it was a peak performer!

Total time: 2.24 sec.


Input tokens: 187, Output tokens: 17


--

KeyboardInterrupt: 

In [15]:
avg_time = 10
avg_in_tok = 103
avg_out_tok = 30

#avg_time = 4.5
#avg_in_tok = 103
#avg_out_tok = 25

# Number of runs
n = 1 # Change to 5 for all the other tests
num_words = len(words)
total_prompts = n*num_words

example_counts = [1, 5, 10, 20, 30]
multiply = [1, 3]#, 5] # 5 did not work yet, because GPU 1 had too much in memory. Maybe try with other GPU map

num_examples = 30


for multiply_by in multiply:

    total_input_tokens = 0
    total_output_tokens = 0
    max_length = num_examples * multiply_by * 30

    print("="*10 + f" INFERENCE TEST with {num_examples * multiply_by} EXAMPLES " + "="*10 + 
        "\n\n" + 
        f"""
        Starting Test with {n} Runs and {num_words} Words/Run. \n
        Total Prompts: {total_prompts}\n\n
        Expected Runtime: {(total_prompts*avg_time)/60}min \n
        Expected Input Tokens: {avg_in_tok*total_prompts} \n
        Expected Output Tokens: {avg_out_tok*total_prompts} \n\n
        """)

    tracker = EmissionsTracker(project_name=f'{model_name}-emissiondata-{num_examples * multiply_by}-examples', log_level="error", pue = 1.22)
    tracker.start()

    # Run the function for each word in the array
    for i in range(n):
        print("="*25 + f" Run {i+1} " + "="*25)
        print("\n")
        for idx, word in enumerate(words):
            print("-"*15 + f" Run {i+1} - Word {idx+1} " + "-"*15)
            output, input_token_count, output_token_count = query_model(word, max_length=max_length, num_examples=num_examples, temperature=0.2, multiply_by=multiply_by)
            
            total_input_tokens += input_token_count
            total_output_tokens += output_token_count
            
            print(output)
            print("\n")
            print(f"Input tokens: {input_token_count}, Output tokens: {output_token_count}")
            print("\n")
        print("\n")
    
    emissions: float = tracker.stop()

    em_i = emissions/total_input_tokens
    em_o = emissions/total_output_tokens
    em_p = emissions/total_prompts


    print("="*15 + f" RESULTS for {num_examples * multiply_by} EXAMPLES " + "="*15 + 
        "\n\n" + 
        f"""
        Finished {n} Runs with {num_words} Words/Run.\n\n
        Total Prompts: {total_prompts}\n
        Total Input Tokens: {total_input_tokens}, AVG/Prompt: {total_input_tokens/total_prompts}\n
        Total Output Tokens: {total_output_tokens}, AVG/Prompt: {total_output_tokens/total_prompts}\n
        """ + 
        
        "-"*50 + "\n" +
        
        f"""
        Total Inference Emissions: {emissions:.3f}kg CO₂eq\n\n
        Emissions / 1.000.000 Input Tokens: {em_i * 1000000:.3f}kg CO₂eq\n
        Emissions / 1.000.000 Output Tokens: {em_o * 1000000:.3f}kg CO₂eq\n
        Emissions / 1.000 Prompts: {em_p * 1000:.3f}kg CO₂eq\n

        """
        )

    # Save results to a CSV file
    results = [
        ["Runs", n],
        ["Words per Run", num_words],
        ["Total Prompts", total_prompts],
        ["Total Input Tokens", total_input_tokens],
        ["Avg Input Tokens per Prompt", round(total_input_tokens / total_prompts, 3)],
        ["Total Output Tokens", total_output_tokens],
        ["Avg Output Tokens per Prompt", round(total_output_tokens / total_prompts, 3)],
        ["Total Emissions (kg CO₂eq)", round(emissions, 3)],
        ["Emissions per Million Input Tokens (kg CO₂eq)", round(em_i * 1000000, 3)],
        ["Emissions per Million Output Tokens (kg CO₂eq)", round(em_o * 1000000, 3)],
        ["Emissions per Thousand Prompts (kg CO₂eq)", round(em_p * 1000, 3)]
    ]

    # Ensure the directory exists
    output_file_path = f'{model_name}-emissiondata-{num_examples * multiply_by}-examples.csv'
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

    with open(output_file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Metric", "Value"])
        writer.writerows(results)

    print(f"Results saved to {output_file_path}")

========== INFERENCE TEST with 30 EXAMPLES ==========


        Starting Test with 1 Runs and 50 Words/Run. 

        Total Prompts: 50


        Expected Runtime: 8.333333333333334min 

        Expected Input Tokens: 5150 

        Expected Output Tokens: 1500 


        


========================= Run 1 =========================


--------------- Run 1 - Word 1 ---------------
Question: Now, tell me 30 short joke(s) for the word: "apple"
 Answer:

Why did the apple join the gym? To get some core strength!
Why did the apple go to the doctor? It had a little core issue!
Why did the apple get kicked out of the bar? It was making too many corny jokes!
Why did the apple go to the beauty parlor? It wanted a peel-fect makeover!
Why did the apple go to the therapist? It was feeling a little crushed!
Why did the apple go to the art museum? To see the core collection!
Why did the apple go to the dance party? To get some juice!
Why did the apple go to the spa? To get a little pressed!
Why did the apple go to the fashion show? To show off its crisp style!
Why did the apple go to the comedy club? To get some laughs and a little crunch!
Why did the apple go to the therapist again? It was feeling a little cored!
Why did the apple go to the gym again? To get some more 

# Approach without Pipeline

In [ ]:
# Write a Text Generation Function
#@track_emissions(project_name=model_name)
#def generate_text(prompt, max_length=50):
#    inputs = tokenizer(prompt, return_tensors="pt").to(device)
#    with torch.no_grad():
#        outputs = model.generate(
#            **inputs, 
#            #max_length=max_length,
#            pad_token_id=tokenizer.eos_token_id,
#            max_new_tokens=max_length
#        )
#    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
#def create_prompt(word): 
#    prompt = f"""
#    You are an AI-Assistant that tells jokes for a given word.
#    Generate a short joke that is exactly 2 sentences long.
#    Do not include any follow-up questions or explanations.#
#
#    For example, a joke for the word "yogurt":
#    'Why did the yogurt go to the art gallery? Because it wanted to be cultured!'#
#
#    Now, tell me one short joke for the word "{word}".
#    """
#    return prompt

In [ ]:
#tracker = EmissionsTracker(project_name=model_name, log_level="error", pue = 1.22)#, output_file=f'{model_name}_emissions.csv')
#tracker.start()

# Number of runs
#n = 1

# Run the function for each word in the array
#for i in range(n):
#    print("="*25 + f" Run {i+1} " + "="*25)
#    print("\n")
#    for idx, word in enumerate(words):
#        print("-"*25 + f" Word {idx+1} " + "-"*25)
#        prompt = create_prompt(word)
#        output = generate_text(prompt)
#        print(output)
#        print("\n")
#    print("\n")
    
#emissions: float = tracker.stop()
#print(emissions)